In [3]:
# This file is used to perform the error class analysis on the structured and unstructured explanations by the LLM

import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()



prompt_name = 'error_class_LLM_nounderover'
prompt = p.error_class_LLM_nounderover

file = "structured_unstructured_dynamic_few_shot_pipeline_4-turbo-preview"

DATA_PATH = f'../../Datasets/Evaluations/Regression/Error_Analysis/structured_unstructured/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Regression/Error_Analysis/error_class_LLM_analysis.csv"

MODEL = "gpt-4-turbo-preview"
TEMP = 0

struct_unstruct = 'unstructured_analysis'





In [4]:
# Define error and correct subsets

df = pd.read_csv(DATA_PATH)

df_correct = df[df['error'] == 'correct']
df_correct_samples = df_correct.sample(n=5, random_state=42)

df_error = df[df['error'] == 'error']
df_error_samples = df_error.sample(n=20, random_state=42)

print(df_correct_samples.shape, df_error_samples.shape)
print(df_correct_samples.head())
print(df_error_samples.head())

df_answer = pd.read_csv(OUTPUT_PATH)



(5, 30) (20, 30)
     Unnamed: 0                         name  year  selling_price  km_driven  \
236         236  Maruti Wagon R LXI Optional  2016         350000      25000   
383         383          Honda City i DTEC S  2015         600000      70000   
301         301          Maruti Alto K10 VXI  2015         300000      24000   
260         260       Hyundai EON Magna Plus  2014         280000      60000   
81           81  Hyundai Grand i10 AT Sportz  2014         374000      54043   

       fuel seller_type transmission         owner     mileage  ...  \
236  Petrol  Individual       Manual   First Owner  20.51 kmpl  ...   
383  Diesel  Individual       Manual   First Owner   26.0 kmpl  ...   
301  Petrol  Individual       Manual   First Owner  23.95 kmpl  ...   
260  Petrol  Individual       Manual  Second Owner   21.1 kmpl  ...   
81   Petrol      Dealer    Automatic   First Owner   18.9 kmpl  ...   

                                      example_prompt_3 example_score_3  \
2

In [5]:
# Define the function to execute the LLM based on the dataset, prompt and Model

def execute(df, prompt, df_answer):
    i = 1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=1000)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=1000)
    examples = ""
    examples += "False Predictions:\n"
    for j in range(len(df_error_samples)):
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_error_samples.iloc[j]['prompt'] + "\n"
        examples += "Prediction: " + str(df_error_samples.iloc[j]['prediction']) + "\n"
        examples += "Explanation: " + df_error_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Actual Price: " + str(df_error_samples.iloc[j]['selling_price']) + "\n"
        i += 1
    examples += "\nCorrect Predictions (Only as a reference):\n"
    for j in range(len(df_correct_samples)):
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_correct_samples.iloc[j]['prompt'] + "\n"
        examples += "Prediction: " + str(df_correct_samples.iloc[j]['prediction']) + "\n"
        examples += "Explanation: " + df_correct_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Actual Price: " + str(df_correct_samples.iloc[j]['selling_price']) + "\n"
        i += 1
    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
    try: 
        result = chain.run(examples = examples, callbacks=[handler])
    except Exception as e:
        print(e)
        print(chain.prompt.format_prompt(examples = examples).to_string())
    handler.langfuse.flush()
    prompt_text = chain.prompt.format_prompt(examples = examples).to_string()
    print(prompt_text)
    print("\n")
    print(result)
    print("\n")
    new_row = {'Prompt_Name': prompt_name, 'Prompt': prompt_text, 'Model': MODEL, 'Database': file, 'Answer': result, 'Context': struct_unstruct}
    df_new_row = pd.DataFrame([new_row])
    df_answer = pd.concat([df_answer, df_new_row])
    return result, df_answer


answer, df_answer = execute(df, prompt, df_answer)
    



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In the following I will give you a few price prediction tasks together with a prediction decision, details about the decision and the actual price.
A wrong prediction is a prediction that deviates by more than 20 percent from the actual price. The Prediction was made by an LLM.
Can you please group the wrong decisions into distinct fault categories? Please also indicate how often each one occurs.
There are also some correct decisions (deviation of less than 20 percent) in the examples. Please just use them as a reference and don't categorize them.
Please go deeper with your analysis and don't use Overestimation or Underestimation as categories.
False Predictions:
Task 1:
Prompt: Example 1:
Features:
name: Maruti 800 AC BSII,
year: 2007,
km_driven: 60000,
fuel: Petrol,
seller_type: Individual,
transmission: Manual,
owner: Second Owner,
mileage: 16.1 kmpl,
engine: 796 CC,
max_power: 37 bhp,
torque: 59Nm@ 2500rpm,
seats: 4.0

Output: "price": 125000 INR 

Example 2:
Features:
name: Maruti

In [6]:
for i in df_answer.values:
    print(i)
    print(i[4])

['error_class_ML'
 "In the following I will give you a few price prediction tasks together with a prediction decision, details about the decision and the actual price.\nA wrong prediction is a prediction that deviates by more than 20 percent from the actual price. \nThe decision was made by a Linear Regression Machine Learning Model and the additional information shows the weights the model assigned to the different tokens. \nCan you please group the wrong decisions into distinct fault categories? Please also indicate how often each one occurs.\nThere are also some correct decisions (deviation of less than 20 percent) in the examples. Please just use them as a reference and don't categorize them.\nPlease go deeper with your analysis and don't just use Overestimation or Underestimation as categories.\nFalse Predictions:\nTask 1:\nAttributes of the car: \nname: Ford Endeavour 3.0L 4X2 AT, year: 8, km_driven: 110000, fuel: Diesel, seller_type: Individual, transmission: Automatic, owner: F

In [48]:
# Save the answer to the output path

df_answer.to_csv(OUTPUT_PATH, index=False)